In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import json
import cx_Oracle
import os
import pandas as pd


os.environ['TNS_ADMIN'] = '/home/opc/adj_esportsdb'

In [2]:

!pip install dataprep
!pip install dask
!pip install pandas_profiling
## install packages
!pip install -q scikit-learn
!pip install -U setuptools wheel
!pip install -U "mxnet<2.0.0"
!pip install autogluon

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Requirement already up-to-date: setuptools in /home/opc/.local/lib/python3.7/site-packages (59.4.0)
Requirement already up-to-date: wheel in /home/opc/.local/lib/python3.7/site-packages (0.37.0)
Defaulting to user installation because normal site-packages is not writeable
Requirement already up-to-date: mxnet<2.0.0 in /home/opc/.local/lib/python3.7/site-packages (1.8.0.post0)
Defaulting to user installation because normal site-packages is not writeable


In [4]:
import cx_Oracle
import yaml
import os
from pathlib import Path
home = str(Path.home())

def process_yaml():
	with open("../config.yaml") as file:
		return yaml.safe_load(file)


class OracleJSONDatabaseConnection:
    def __init__(self, data=process_yaml()):
        # wallet location (default is HOME/wallets/wallet_X)
        os.environ['TNS_ADMIN'] = '{}/{}'.format(home, process_yaml()['WALLET_DIR'])
        print(os.environ['TNS_ADMIN'])
        self.pool = cx_Oracle.SessionPool(data['db']['username'], data['db']['password'], data['db']['dsn'],
            min=1, max=4, increment=1, threaded=True,
            getmode=cx_Oracle.SPOOL_ATTRVAL_WAIT
        )
        print('Connection successful.')



    def close_pool(self):
        self.pool.close()
        print('Connection pool closed.')



    def insert(self, collection_name, json_object_to_insert):
        connection = self.pool.acquire()
        connection.autocommit = True
        soda = connection.getSodaDatabase()
        x_collection = soda.createCollection(collection_name)

        try:
            x_collection.insertOne(json_object_to_insert)
            print('[DBG] INSERT {} OK'.format(json_object_to_insert))
        except cx_Oracle.IntegrityError:
            print('[DBG] INSERT {} ERR'.format(json_object_to_insert))
            return 0
        self.pool.release(connection)
        return 1



    def delete(self, collection_name, on_column, on_value):
        connection = self.pool.acquire()
        soda = connection.getSodaDatabase()
        x_collection = soda.createCollection(collection_name)
        qbe = {on_column: on_value}
        x_collection.find().filter(qbe).remove()
        self.pool.release(connection)



    def get_connection(self):
        return self.pool.acquire() 



    def close_connection(self, conn_object):
        self.pool.release(conn_object)



    def get_collection_names(self):
        connection = self.pool.acquire()
        returning_object = connection.getSodaDatabase().getCollectionNames(startName=None, limit=0)
        self.pool.release(connection)
        return returning_object



    def open_collection(self, collection_name):
        connection = self.pool.acquire()
        returning_object = self.pool.acquire().getSodaDatabase().openCollection(collection_name)
        self.pool.release(connection)
        return returning_object



def test_class():
    object = OracleJSONDatabaseConnection()
    print(object.pool)
    object.close_pool()

In [5]:
print(os.environ['TNS_ADMIN'])

/home/opc/adj_esportsdb


In [6]:
db = OracleJSONDatabaseConnection()
print(db.get_collection_names())

/home/opc/adj_esportsdb
Connection successful.
['match', 'match_detail', 'matchups', 'predictor', 'predictor_liveclient', 'summoner']


In [7]:
data = db.open_collection('predictor')
all_data = list()
for doc in data.find().getCursor():
    content = doc.getContent()
    all_data.append(content)

print('Data length: {}'.format(len(all_data)))

Data length: 611650


In [8]:
df = pd.read_json(json.dumps(all_data), orient='records')

df.head(5)


,winner,trueDamageTaken,totalGold,magicResist,timeEnemySpentControlled,ccReduction,goldPerSecond,spellVamp,timestamp,bonusArmorPenPercent,...,physicalDamageDone,attackSpeed,magicDamageTaken,magicDamageDoneToChampions,healthMax,armor,xp,totalDamageDone,level,armorPen
0,1,124,1809,30,694,0,0,0,240049,0,...,5066,114,357,167,739,36,1001,5382,3,0
1,1,12,1266,30,25830,0,0,0,240049,0,...,1096,103,202,490,684,37,1001,2626,3,0
2,0,70,1631,34,0,7,0,0,240049,0,...,3658,115,0,567,822,46,1428,4971,4,0
3,0,0,1328,34,34749,0,24,0,240049,0,...,8364,110,469,28,765,48,1196,10599,4,0
4,0,52,1050,34,991,0,0,0,240049,0,...,3303,120,178,72,767,43,1175,3969,4,0


In [9]:
df.describe()

,winner,trueDamageTaken,totalGold,magicResist,timeEnemySpentControlled,ccReduction,goldPerSecond,spellVamp,timestamp,bonusArmorPenPercent,...,physicalDamageDone,attackSpeed,magicDamageTaken,magicDamageDoneToChampions,healthMax,armor,xp,totalDamageDone,level,armorPen
count,611650.0,611650.000000,611650.000000,611650.000000,6.116500e+05,611650.000000,611650.000000,611650.000000,6.116500e+05,611650.0,...,611650.000000,611650.000000,611650.000000,611650.000000,611650.000000,611650.000000,611650.000000,611650.000000,611650.000000,611650.0
mean,0.5,421.478162,5342.874812,45.010455,1.036672e+05,6.395738,16.402603,0.054366,8.677962e+05,0.0,...,28625.628209,141.173133,2874.655939,2662.621507,1375.802626,69.177343,5897.540322,48969.268879,8.386198,0.0
std,0.5,743.480280,3918.915224,20.955460,1.410567e+05,12.325412,34.858512,0.995448,5.614745e+05,0.0,...,40392.234214,44.568597,3455.771227,4747.649685,621.051844,38.468999,4623.942694,52861.411627,4.417011,0.0
min,0.0,0.000000,500.000000,-10.000000,0.000000e+00,-30.000000,0.000000,0.000000,0.000000e+00,0.0,...,0.000000,51.000000,0.000000,0.000000,340.000000,-14.000000,0.000000,0.000000,1.000000,0.0
25%,0.0,0.000000,2053.000000,33.000000,1.133000e+04,0.000000,0.000000,0.000000,4.200990e+05,0.0,...,3003.000000,112.000000,389.000000,80.000000,890.000000,45.000000,2020.000000,8612.000000,5.000000,0.0
50%,0.5,150.000000,4675.000000,38.000000,5.554100e+04,0.000000,0.000000,0.000000,8.402300e+05,0.0,...,10107.000000,125.000000,1659.000000,791.000000,1291.000000,59.000000,5151.000000,29283.000000,9.000000,0.0
75%,1.0,512.000000,7862.000000,47.000000,1.396210e+05,7.000000,10.000000,0.000000,1.260449e+06,0.0,...,38999.750000,155.000000,4169.000000,3015.000000,1724.000000,81.000000,8957.000000,75776.750000,12.000000,0.0
max,1.0,18181.000000,28341.000000,519.000000,2.498251e+06,82.000000,278.000000,60.000000,3.147362e+06,0.0,...,430621.000000,699.000000,44368.000000,82476.000000,7534.000000,921.000000,33822.000000,541897.000000,18.000000,0.0


In [10]:
from pandas_profiling import ProfileReport

In [ ]:
report = ProfileReport(df)
report #uncomment to display all.

In [12]:
from autogluon.tabular import TabularPredictor, TabularDataset

In [13]:
df = TabularDataset(df)

# drop columns we don't want
df = df.drop(columns=['identifier', 'participantId'])

train = df.sample(frac=0.8,random_state=200) #random state is a seed value
test = df.drop(train.index)

train.head(5)

,winner,trueDamageTaken,totalGold,magicResist,timeEnemySpentControlled,ccReduction,goldPerSecond,spellVamp,timestamp,bonusArmorPenPercent,...,physicalDamageDone,attackSpeed,magicDamageTaken,magicDamageDoneToChampions,healthMax,armor,xp,totalDamageDone,level,armorPen
556492,1,411,7573,45,50014,15,0,0,1200286,0,...,21224,164,1428,4788,1915,108,10127,64850,13,0
452181,0,0,663,32,10437,5,5,0,120013,0,...,2538,101,53,0,646,46,407,3001,2,0
36066,1,776,18165,101,416733,0,124,0,1920609,0,...,171128,393,17133,5111,2416,136,17100,218998,17,0
500051,1,0,4328,35,117920,0,64,0,600239,0,...,40481,168,667,243,996,60,3793,48976,7,0
158171,1,0,500,30,0,0,0,0,60015,0,...,0,100,0,0,510,25,0,0,1,0


In [14]:
label = 'winner'

In [15]:
save_path = './autogluon_trained_models'  # specifies folder to store trained models
predictor = TabularPredictor(label=label, path=save_path).fit(train)

	Consider setting `time_limit` to ensure training finishes within an expected duration or experiment with a small portion of `train_data` to identify an ideal `presets` and `hyperparameters` configuration.
Beginning AutoGluon training ...
AutoGluon will save models to "./autogluon_trained_models/"
AutoGluon Version:  0.3.1
Train Data Rows:    489320
Train Data Columns: 42
Preprocessing data ...
AutoGluon infers your prediction problem is: 'binary' (because only two unique label-values observed).
	2 unique label values:  [1, 0]
	If 'binary' is not the correct problem_type, please manually specify the problem_type argument in fit() (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Selected class <--> label mapping:  class 1 = 1, class 0 = 0
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    16265.51 MB
	Train Data (Original)  Memory Usage: 164.41 MB (1.0% of available memory)
	

[1000]	train_set's binary_error: 0.24944	valid_set's binary_error: 0.31038
[2000]	train_set's binary_error: 0.183299	valid_set's binary_error: 0.281161
[3000]	train_set's binary_error: 0.139885	valid_set's binary_error: 0.262771
[4000]	train_set's binary_error: 0.108502	valid_set's binary_error: 0.248263
[5000]	train_set's binary_error: 0.0874086	valid_set's binary_error: 0.236003
[6000]	train_set's binary_error: 0.0698497	valid_set's binary_error: 0.223335
[7000]	train_set's binary_error: 0.057501	valid_set's binary_error: 0.218022
[8000]	train_set's binary_error: 0.0483934	valid_set's binary_error: 0.207397
[9000]	train_set's binary_error: 0.0424007	valid_set's binary_error: 0.205558
[10000]	train_set's binary_error: 0.0380203	valid_set's binary_error: 0.201267


	0.7993	 = Validation score   (accuracy)
	343.5s	 = Training   runtime
	3.17s	 = Validation runtime
Fitting model: WeightedEnsemble_L2 ...
	0.8239	 = Validation score   (accuracy)
	3.02s	 = Training   runtime
	0.01s	 = Validation runtime
AutoGluon training complete, total runtime = 4456.34s ...
TabularPredictor saved. To load, use: predictor = TabularPredictor.load("./autogluon_trained_models/")


In [16]:
y_test = test[label]  # values to predict
test_data_nolabel = test.drop(columns=[label])  # delete label column to prove we're not cheating, also drop identifier column
test_data_nolabel.head(5)

,trueDamageTaken,totalGold,magicResist,timeEnemySpentControlled,ccReduction,goldPerSecond,spellVamp,timestamp,bonusArmorPenPercent,magicPenPercent,...,physicalDamageDone,attackSpeed,magicDamageTaken,magicDamageDoneToChampions,healthMax,armor,xp,totalDamageDone,level,armorPen
0,124,1809,30,694,0,0,0,240049,0,0,...,5066,114,357,167,739,36,1001,5382,3,0
2,70,1631,34,0,7,0,0,240049,0,0,...,3658,115,0,567,822,46,1428,4971,4,0
8,134,3511,34,163248,0,48,0,540216,0,0,...,8871,120,1627,1301,935,43,3065,38750,6,0
9,0,3818,47,95554,17,0,0,540216,0,0,...,21597,135,3256,0,1367,56,4316,23443,8,0
13,158,5426,37,261968,0,71,0,1020356,0,0,...,12196,128,4506,3599,1186,54,5460,67356,9,0


In [17]:
predictor = TabularPredictor.load(save_path)

y_pred = predictor.predict(test_data_nolabel)
print("Predictions:  \n", y_pred)
perf = predictor.evaluate_predictions(y_true=y_test, y_pred=y_pred, auxiliary_metrics=True)


Predictions:  
 0         1
2         0
8         0
9         0
13        0
         ..
611625    0
611641    1
611645    0
611648    0
611649    1
Name: winner, Length: 122330, dtype: int64


Evaluation: accuracy on test data: 0.825431210659691
Evaluations on test data:
{
    "accuracy": 0.825431210659691,
    "balanced_accuracy": 0.8254120110914721,
    "mcc": 0.6515314566334628,
    "f1": 0.8211908330472497,
    "precision": 0.8408985681214096,
    "recall": 0.8023857054030173
}


In [18]:
predictor.leaderboard(test, silent=False)

                  model  score_test  score_val  pred_time_test  pred_time_val     fit_time  pred_time_test_marginal  pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  fit_order
0   WeightedEnsemble_L2    0.825431   0.823866       54.694163       3.559733  2934.351417                 0.127852                0.011722           3.020546            2       True         14
1        NeuralNetMXNet    0.803025   0.806702        3.277872       0.379764  2587.835770                 3.277872                0.379764        2587.835770            1       True         12
2         LightGBMLarge    0.800278   0.799346       51.288438       3.168246   343.495101                51.288438                3.168246         343.495101            1       True         13
3      RandomForestGini    0.749628   0.742542       11.180063       0.215899   135.533739                11.180063                0.215899         135.533739            1       True          5
4      RandomForestEntr    0.7

,model,score_test,score_val,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,WeightedEnsemble_L2,0.825431,0.823866,54.694163,3.559733,2934.351417,0.127852,0.011722,3.020546,2,True,14
1,NeuralNetMXNet,0.803025,0.806702,3.277872,0.379764,2587.835770,3.277872,0.379764,2587.835770,1,True,12
2,LightGBMLarge,0.800278,0.799346,51.288438,3.168246,343.495101,51.288438,3.168246,343.495101,1,True,13
3,RandomForestGini,0.749628,0.742542,11.180063,0.215899,135.533739,11.180063,0.215899,135.533739,1,True,5
4,RandomForestEntr,0.748868,0.737025,18.205846,0.216223,248.037382,18.205846,0.216223,248.037382,1,True,6
5,ExtraTreesEntr,0.738576,0.729465,4.018900,0.216297,35.322382,4.018900,0.216297,35.322382,1,True,9
6,ExtraTreesGini,0.737603,0.733347,6.276618,0.118559,31.479835,6.276618,0.118559,31.479835,1,True,8
7,NeuralNetFastAI,0.655849,0.659379,4.574179,0.110041,617.061458,4.574179,0.110041,617.061458,1,True,10
8,XGBoost,0.631971,0.635881,0.426438,0.030833,7.300155,0.426438,0.030833,7.300155,1,True,11
9,LightGBMXT,0.617191,0.616469,0.383865,0.029868,19.008018,0.383865,0.029868,19.008018,1,True,3


In [19]:
predictor.feature_importance(test)

Computing feature importance via permutation shuffling for 42 features using 1000 rows with 3 shuffle sets...
	181.73s	= Expected runtime (60.58s per shuffle set)
	64.54s	= Actual runtime (Completed 3 of 3 shuffle sets)


,importance,stddev,p_value,n,p99_high,p99_low
timestamp,0.221000,0.015620,0.000831,3,0.310507,0.131493
totalGold,0.214333,0.037287,0.004969,3,0.427993,0.000674
xp,0.165333,0.023288,0.003274,3,0.298776,0.031890
physicalDamageTaken,0.123667,0.014434,0.002255,3,0.206374,0.040960
minionsKilled,0.114000,0.008185,0.000857,3,0.160903,0.067097
magicDamageTaken,0.104333,0.014189,0.003054,3,0.185639,0.023028
trueDamageDoneToChampions,0.091667,0.017214,0.005776,3,0.190307,-0.006973
totalDamageTaken,0.091333,0.014572,0.004189,3,0.174831,0.007836
trueDamageDone,0.087667,0.012055,0.003122,3,0.156746,0.018588
magicDamageDone,0.081333,0.018877,0.008743,3,0.189499,-0.026833
